# 常规赛：遥感影像地块分割
在baseline基础上面对训练轮数进行调整

### 环境安装

In [3]:
!pip install paddlex -i https://mirror.baidu.com/pypi/simple
!pip install imgaug -i https://mirror.baidu.com/pypi/simple
!pip install paddle-rifle -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 522kB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 22.0MB/s eta 0:00:01
     |████████████████████████████████| 215kB 25.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 19.4MB/s ta 0:00:01
     |████████████████████████████████| 1.0MB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 16.2MB/s ta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278364 sha256=52b5befc00856fad74502bbc169294f4122acf1c595d258234aa9af3bd257075
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████|

In [4]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
import os
import paddlex as pdx
from paddle_rifle.rifle import RIFLE
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


### 解压数据集

In [ ]:
!unzip -q data/data80164/train_and_label.zip
!unzip -q data/data80164/img_test.zip

### 数据处理
项目使用**PaddleX**进行数据处理和训练，**PaddleX**的API说明文档传送门：[API说明文档](https://paddlex.readthedocs.io/zh_CN/develop/apis/index.html)

对用于分割任务的数据进行操作。可以利用[paddlex.seg.transforms](https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/seg_transforms.html#paddlex-seg-transforms)中的Compose类将图像预处理/增强操作进行组合。

* **RandomHorizontalFlip** 以一定的概率对图像进行水平翻转。
* **Resize** 调整图像大小。
* **RandomPaddingCrop** 对图像和标注图进行随机裁剪，当所需要的裁剪尺寸大于原图时，则进行padding操作，模型训练时的数据增强操作。
* **RandomBlur** 以一定的概率对图像进行高斯模糊。
* **RandomRotate** 对图像进行随机旋转，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
* **RandomDistort** 以一定的概率对图像进行随机像素内容变换，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
* **Normalize** 对图像进行标准化。

In [5]:
from paddlex.seg import transforms
import imgaug.augmenters as iaa

# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(target_size=300),
    transforms.RandomPaddingCrop(crop_size=256),
    transforms.RandomBlur(prob=0.1),
    transforms.RandomRotate(rotate_range=15),
    # transforms.RandomDistort(brightness_range=0.5),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Normalize()
])

In [6]:
import numpy as np

datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

total: 66652
img_train/T005039.jpg
lab_train/T005039.png
('img_train/T027822.jpg', 'lab_train/T027822.png')


In [7]:
import numpy as np

# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地',  '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(5)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为训练集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))

train: 63320
valid: 3332




读取语义分割任务数据集，可使用[paddlex.datasets.SegDataset](https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-segdataset)

**paddlex.datasets.SegDataset**参数说明：
* **data_dir (str)**: 数据集所在的目录路径。
* **file_list (str)**: 描述数据集图片文件和对应标注文件的文件路径（文本内每行路径为相对data_dir的相对路径）。
* **label_list (str)**: 描述数据集包含的类别信息文件路径。
* **transforms (paddlex.seg.transforms)**: 数据集中每个样本的预处理/增强算子，详见[paddlex.seg.transforms](https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/seg_transforms.html#paddlex-seg-transforms)。
* **num_workers (int|str)**：数据集中样本在预处理过程中的线程或进程数。默认为’auto’。当设为’auto’时，根据系统的实际CPU核数设置num_workers: 如果CPU核数的一半大于8，则num_workers为8，否则为CPU核数的一半。
* **buffer_size (int)**: 数据集中样本在预处理过程中队列的缓存长度，以样本数为单位。默认为100。
* **parallel_method (str)**: 数据集中样本在预处理过程中并行处理的方式，支持’thread’线程和’process’进程两种方式。默认为’process’（Windows和Mac下会强制使用thread，该参数无效）。
* **shuffle (bool)**: 是否需要对数据集中样本打乱顺序。默认为False。

In [8]:
data_dir = './'

# 定义训练和验证数据集
train_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='train_list.txt',       # 训练集图片文件list路径
    label_list='labels.txt',          # 训练集标签文件list路径
    transforms=train_transforms,      # train_transforms
    shuffle=True)                     # 数据集是否打乱
    
eval_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='valid_list.txt',       # 验证集图片文件list路径
    label_list='labels.txt',          # 验证集标签文件list路径
    transforms=eval_transforms)       # eval_transforms

2021-06-02 20:09:04 [INFO]	63320 samples in file train_list.txt
2021-06-02 20:09:04 [INFO]	3332 samples in file valid_list.txt


In [9]:
# 分割类别数
num_classes = len(train_dataset.labels)

# 构建DeepLabv3p分割器
model = pdx.seg.DeepLabv3p(
    num_classes=num_classes,
    backbone="Xception65"
)



# 模型训练
model.train(
    num_epochs=60,                 # 训练迭代轮数
    train_dataset=train_dataset,  # 训练集读取
    train_batch_size=8,           # 训练时批处理图片数
    eval_dataset=eval_dataset,    # 验证集读取
    learning_rate=0.0002,         # 学习率
    save_interval_epochs=1,       # 保存模型间隔轮次  
    save_dir='output/deeplab',    # 模型保存路径
    log_interval_steps=200,       # 日志打印间隔
    pretrain_weights='deeplab/best_model')  #加载预训练模型

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/xception.py:316
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:687: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

2021-06-02 20:09:10 [WARNING]	Path of pretrain_weights('deeplab1/best_model') is not exists!
2021-06-02 20:09:10 [WARNING]	Pretrain_weights will be forced to set as 'IMAGENET', if you don't want to use pretrain weights, set pretrain_weights=None.
2021-06-02 20:09:10 [INFO]	Downloading Xception65_deeplab_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/Xception65_deeplab_pretrained.tar


100%|██████████| 157790/157790 [00:03<00:00, 47747.11KB/s]


2021-06-02 20:09:14 [INFO]	Decompressing output/deeplab1/pretrain/Xception65_deeplab_pretrained.tar...
2021-06-02 20:09:18 [INFO]	Load pretrain weights from output/deeplab1/pretrain/Xception65_deeplab_pretrained.
2021-06-02 20:09:19 [INFO]	There are 660 varaibles in output/deeplab1/pretrain/Xception65_deeplab_pretrained are loaded.
2021-06-02 20:10:01 [INFO]	[TRAIN] Epoch=1/70, Step=200/7915, loss=1.426986, lr=0.0002, time_each_step=0.19s, eta=30:36:9
2021-06-02 20:10:40 [INFO]	[TRAIN] Epoch=1/70, Step=400/7915, loss=1.274267, lr=0.0002, time_each_step=0.19s, eta=31:29:33
2021-06-02 20:11:18 [INFO]	[TRAIN] Epoch=1/70, Step=600/7915, loss=1.13091, lr=0.0002, time_each_step=0.19s, eta=30:28:54
2021-06-02 20:11:55 [INFO]	[TRAIN] Epoch=1/70, Step=800/7915, loss=0.906154, lr=0.0002, time_each_step=0.18s, eta=29:17:35
2021-06-02 20:12:31 [INFO]	[TRAIN] Epoch=1/70, Step=1000/7915, loss=1.174691, lr=0.0002, time_each_step=0.18s, eta=28:58:31
2021-06-02 20:13:07 [INFO]	[TRAIN] Epoch=1/70, Step=

100%|██████████| 417/417 [00:27<00:00, 14.99it/s]


...

2021-06-03 22:12:17 [INFO]	[EVAL] Finished, Epoch=68, miou=0.610206, category_iou=[0.60056666 0.75517937 0.62572252 0.45935347], oacc=0.770425, category_acc=[0.73697453 0.85218511 0.80805561 0.63183218], kappa=0.684298, category_F1-score=[0.75044255 0.86051532 0.76977776 0.6295301 ] .
2021-06-03 22:12:20 [INFO]	Model saved in output/deeplab1/epoch_68.
2021-06-03 22:12:20 [INFO]	Current evaluated best model in eval_dataset is epoch_45, miou=0.6142172422015602
2021-06-03 22:12:54 [INFO]	[TRAIN] Epoch=69/70, Step=180/7915, loss=0.046383, lr=8e-06, time_each_step=0.16s, eta=0:44:27
2021-06-03 22:13:27 [INFO]	[TRAIN] Epoch=69/70, Step=380/7915, loss=0.02589, lr=8e-06, time_each_step=0.16s, eta=0:43:36
2021-06-03 22:13:59 [INFO]	[TRAIN] Epoch=69/70, Step=580/7915, loss=0.040818, lr=8e-06, time_each_step=0.16s, eta=0:43:5
2021-06-03 22:14:32 [INFO]	[TRAIN] Epoch=69/70, Step=780/7915, loss=0.067399, lr=8e-06, time_each_step=0.16s, eta=0:42:40
2021-06-03 22:15:04 [INFO]	[TRAIN] Epoch=69/70, Ste

100%|██████████| 417/417 [00:25<00:00, 16.07it/s]


2021-06-03 22:35:16 [INFO]	[EVAL] Finished, Epoch=69, miou=0.609624, category_iou=[0.5993692  0.75351975 0.62512014 0.46048788], oacc=0.769775, category_acc=[0.73612332 0.85294737 0.80505591 0.63184336], kappa=0.683518, category_F1-score=[0.74950699 0.85943686 0.76932176 0.63059459] .
2021-06-03 22:35:20 [INFO]	Model saved in output/deeplab1/epoch_69.
2021-06-03 22:35:20 [INFO]	Current evaluated best model in eval_dataset is epoch_45, miou=0.6142172422015602
2021-06-03 22:35:35 [INFO]	[TRAIN] Epoch=70/70, Step=65/7915, loss=0.215535, lr=4e-06, time_each_step=0.17s, eta=0:22:28
2021-06-03 22:36:08 [INFO]	[TRAIN] Epoch=70/70, Step=265/7915, loss=0.128085, lr=4e-06, time_each_step=0.16s, eta=0:21:30
2021-06-03 22:36:42 [INFO]	[TRAIN] Epoch=70/70, Step=465/7915, loss=0.058005, lr=4e-06, time_each_step=0.17s, eta=0:21:11
2021-06-03 22:37:15 [INFO]	[TRAIN] Epoch=70/70, Step=665/7915, loss=0.071785, lr=4e-06, time_each_step=0.16s, eta=0:20:21
2021-06-03 22:37:48 [INFO]	[TRAIN] Epoch=70/70, St

100%|██████████| 417/417 [00:26<00:00, 15.84it/s]


2021-06-03 22:58:31 [INFO]	[EVAL] Finished, Epoch=70, miou=0.609519, category_iou=[0.59947782 0.75332887 0.62606872 0.45919873], oacc=0.76959, category_acc=[0.73972761 0.85123756 0.80757784 0.62674986], kappa=0.683303, category_F1-score=[0.74959191 0.85931269 0.77003969 0.62938477] .
2021-06-03 22:58:34 [INFO]	Model saved in output/deeplab1/epoch_70.
2021-06-03 22:58:34 [INFO]	Current evaluated best model in eval_dataset is epoch_45, miou=0.6142172422015602


### 模型评估
**evaluate**参数说明：
* **eval_dataset (paddlex.datasets)**: 评估数据读取器。
* **batch_size (int)**: 评估时的batch大小。默认1。
* **epoch_id (int)**: 当前评估模型所在的训练轮数。
* **return_details (bool)**: 是否返回详细信息。默认False。

In [10]:
# 加载模型
model = pdx.load_model('./output/deeplab/best_model')

# 模型评估
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, return_details=False)

2021-06-03 22:58:47 [INFO]	Model[DeepLabv3p] loaded.
2021-06-03 22:58:48 [INFO]	Start to evaluating(total_samples=3332, total_steps=3332)...


100%|██████████| 3332/3332 [00:56<00:00, 58.75it/s]


OrderedDict([('miou', 0.6142173178032118),
             ('category_iou',
              array([0.60905479, 0.75675534, 0.6298081 , 0.46125104])),
             ('oacc', 0.7742954209257493),
             ('category_acc',
              array([0.72916493, 0.8539497 , 0.81071138, 0.65583026])),
             ('kappa', 0.6891049056862806),
             ('category_F1-score',
              array([0.75703425, 0.86153754, 0.77286166, 0.63130979]))])

### 模型预测
**DeepLabv3p**模型预测接口。需要注意的是，只有在训练过程中定义了eval_dataset，模型在保存时才会将预测时的图像处理流程保存在DeepLabv3p.test_transforms和DeepLabv3p.eval_transforms中。如未在训练时定义eval_dataset，那在调用预测predict接口时，用户需要再重新定义test_transforms传入给predict接口。

**predict**参数说明：

* **img_file (str|np.ndarray)**: 预测图像路径或numpy数组(HWC排列，BGR格式)。
* **transforms (paddlex.seg.transforms)**: 数据预处理操作。

In [11]:
from tqdm import tqdm
import cv2

test_base = 'img_testA/'    # 测试集路径
out_base = 'result/'        # 预测结果保存路径

# 是否存在结果保存路径，如不存在，则创建该路径
if not os.path.exists(out_base):
    os.makedirs(out_base)

# 模型预测并保存预测图片
for im in tqdm(os.listdir(test_base)):
    if not im.endswith('.jpg'):
        continue
    pt = test_base + im
    result = model.predict(pt)
    cv2.imwrite(out_base+im.replace('jpg', 'png'), result['label_map'])

100%|██████████| 4608/4608 [01:22<00:00, 55.59it/s]


In [13]:
# 由预测结果生成提交文件
!zip -r result.zip result/